# Greedy Algorithm

Author: Felix Hoffmann <br>
Date: 05/10/2024 <br>
Based on: Reduced Basis Methods for Partial Differential Equations - An Introduction

This Tutorial will cover the Greedy Algorithm.

---

We will implement the Greedy Algorithm according to Algorithm 7.3 of the book.

First we will define all the helper functions. Their pseudocode can also found in the book. If modifications were made, they are stated at the according function.

In [1]:
import src.helperNS as helperNS
import src.defineProblem as dP

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
import scipy
import tqdm

<font color='Red'> <b>ToDo:</b> </font> <br>
In the following function we are ignoring the Dirichlet Boundary. Implement these.

In [2]:
def solveHFsystem(AAhq, ffhq, thetaAq, thetaFq, mu_N):
    Qa = len(AAhq)
    Qf = len(ffhq)
    
    AAh = np.zeros_like(AAhq[0])
    ffh = np.zeros_like(ffhq[0])
    
    for q in range(Qa):
        AAh = AAh + thetaAq[q](mu_N) * AAhq[q]
        
    for q in range(Qf):
        ffh = ffh + thetaFq[q](mu_N) * ffhq[q]
        
    u_Sol = scipy.sparse.linalg.spsolve(AAh, ffh)
    
    return u_Sol

In the following we're adding an additional projection step if the norm of the projection is too small. This prevents the loss of orthogonality. 

In [3]:
def normM(z, M):
    return np.sqrt(z @ M @ z)


def gramSchmidt(V, u, X):
    if V.size == 0:
        z = u
    else:
        z = u - V @ V.T @ X @ u

        # cure for loss of orthogonality
        if np.linalg.norm(z) < 0.7*np.linalg.norm(u):
            z = z - V @ V.T @ X @ z
    
    z = z / normM(z, X)
    
    return z

In [4]:
def projectSystem(AAhq, ffhq, V, X, method="G-RB"):
    Qa = len(AAhq)
    Qf = len(ffhq)
    
    if method == "G-RB":
        AANq = [None] * Qa
        ffNq = [None] * Qf
        
        for q in range(Qa):
            AANq[q] = V.T @ AAhq[q] @ V
        
        for q in range(Qf):
            ffNq[q] = V.T @ ffhq[q]
    
    elif method == "LS-RB":
        AANq = [[None for _ in range(Qa)] for _ in range(Qa)]
        ffNq = [[None for _ in range(Qf)] for _ in range(Qa)]
        
        for q1 in range(Qa):
            ZZ = scipy.sparse.linalg.spsolve(X, AAhq[q1] @ V)
            for q2 in range(Qa):
                AANq[q1][q2] = ZZ.T @ AAhq[q2] @ V
            for q2 in range(Qf):
                ffNq[q1][q2] = ZZ.T @ ffhq[q2]
            
    else:
        print("Method not found.")
        return -1

    return AANq, ffNq

In [5]:
def offlineResidual(AAh, ffh, XXh, V_base):
    Qa = len(AAh)
    Qf = len(ffh)
    
    C = [[None for _ in range(Qf)] for _ in range(Qf)]
    d = [[None for _ in range(Qf)] for _ in range(Qa)]
    E = [[None for _ in range(Qa)] for _ in range(Qa)]
    
    for q1 in range(Qf):
        # Matlab solves via Cholesky Decomposition
        t = scipy.sparse.linalg.spsolve(XXh, ffh[q1])
        for q2 in range(Qf):
            C[q1][q2] = t @ ffh[q2]
            
    for q1 in range(Qa):
        # Matlab solves via Cholesky Decomposition
        Z = scipy.sparse.linalg.spsolve(XXh, AAh[q1] @ V_base)
        for q2 in range(Qa):
            EE = Z.T @ AAh[q2] @ V_base
            
            if EE.shape == (1,):
                EE = np.reshape(EE, (1,1))
                
            E[q1][q2] = EE
            
        for q2 in range(Qf):
            dd = Z.T @ ffh[q2]
            
            if dd.shape == ():
                dd = np.reshape(dd, (1,))
                
            d[q1][q2] = dd
        
    return C, d, E

<font color='Red'> <b>ToDo:</b> </font> <br>
In the following function we are ignoring the Dirichlet Boundary. Implement these.

In [6]:
def solveRBsystem(AANq, ffNq, thetaA, thetaF, mu, method):    
    if method == "G-RB":
        Qa = len(AANq)
        Qf = len(ffNq)
    
        AAN = np.zeros_like(AANq[0])
        ffN = np.zeros_like(ffNq[0])

        for q in range(Qa):
            AAN = AAN + thetaA[q](mu) * AANq[q]

        for q in range(Qf):
            ffN = ffN + thetaF[q](mu) * ffNq[q]
        
        uN = np.linalg.solve(AAN, ffN)
    
    
    elif method == "LS-RB":
        Qa = len(AANq)
        Qf = len(ffNq[0])
                
        AAN = np.zeros_like(AANq[0][0])
        ffN = np.zeros_like(ffNq[0][0])
        
        for q1 in range(Qa):
            for q2 in range(Qa):
                AAN = AAN + thetaA[q1](mu)*thetaA[q2](mu) * AANq[q1][q2]
        
        for q1 in range(Qa):
            for q2 in range(Qf):
                ffN = ffN + thetaA[q1](mu)*thetaF[q2](mu) * ffNq[q1][q2]        
        
        if AAN.shape == (1,):
            uN = np.linalg.solve([AAN], [ffN])
        else:
            uN = np.linalg.solve(AAN, ffN)
            
    else:
        print("Method not found.")
        return -1
        
    return uN

In this function, a crucial change is made. We subtract ```res_AF``` instead of adding it as the other two residuals. In the book it's stated as an addition, but the greedy algorithm doesn't work if we add the crossterm residual. 

<font color='Red'> <b>ToDo:</b> </font> <br>
Find out why we need a negative Sign.

In [7]:
def errorEstimate(C, d, EE, thetaA, thetaF, uN, mu, beta):
    epsilon = 0
    
    res_AA = 0
    res_AF = 0
    res_FF = 0
    
    Qa = len(thetaA)
    Qf = len(thetaF)
    
    for q1 in range(Qf):
        for q2 in range(Qf):
            res_FF = res_FF + thetaF[q1](mu)*thetaF[q2](mu) * C[q1][q2]
    
    for q1 in range(Qa):
        for q2 in range(Qa):
            res_AA = res_AA + thetaA[q1](mu)*thetaA[q2](mu) * uN @ EE[q1][q2] @ uN
        for q2 in range(Qf):
            res_AF = res_AF + (thetaA[q1](mu)*thetaF[q2](mu) * uN @ d[q1][q2] + 
                               thetaA[q1](mu)*thetaF[q2](mu) * d[q1][q2] @ uN)

    #! Here we subtract res_AF!
    epsilon = res_AA - res_AF + res_FF
    delta = np.sqrt(np.abs(epsilon)) / beta
    
    return delta

--- 
Additionally we need some helper functions:

In [8]:
def addZeta(V_base, zeta_N):
    if V_base.shape == (0,):
        V_base = np.array(zeta_N).reshape((-1, 1))
    else:
        V_base = np.hstack((V_base, zeta_N.reshape((-1, 1))))

    return V_base


def addMu(xi_g, mu_N):
    if xi_g.shape == (0,):
        xi_g = np.array(mu_N)
    else:
        xi_g = np.vstack((xi_g, mu_N))
    
    return xi_g

In [9]:

def calcXX(mesh):
      import dolfinx.fem as xfem
      import dolfinx.fem.petsc as xpetsc
      import ufl
      import scipy
      
      V = xfem.FunctionSpace(mesh, ("Lagrange", 1))
      v1 = ufl.TrialFunction(V)
      v2 = ufl.TestFunction(V)
      
      xx = xfem.form(v1 * v2 * ufl.dx)
      
      XX = xpetsc.assemble_matrix(xx)
      XX.assemble()
      
      ai, aj, av = XX.getValuesCSR()
      XX_csr = scipy.sparse.csr_matrix((av, aj, ai))
      
      return XX_csr

def overhead():
      mesh, ct, _ = helperNS.readMesh("./mesh/mixer.msh")
      
      XX = scipy.sparse.csc_matrix(calcXX(mesh))
      u, v, b, facet_tag, V, ds, fdim = helperNS.overhead(mesh)
      bc1, bc5, bc6 = helperNS.calcBc(mesh, u, v, facet_tag, V, ds, fdim)
      calcPDE = lambda mu: helperNS.calcPDE(mu, mesh, u, v, b, facet_tag, V, ds, fdim, bc1, bc5, bc6, plotting=False)

      return calcPDE, XX, mesh

In [10]:
def calcGlobalBeta(xi_train, XXh, AAh, thetaA, ffh, thetaF):
    print("Start Calculating Beta")
    beta_mu = np.array([])
    for i in tqdm.tqdm(range(xi_train.shape[0])):
        mu = xi_train[i]
        beta_local = calcBeta(XXh, mu, AAh, thetaA, ffh, thetaF)
        beta_mu = np.append(beta_mu, beta_local)
        # print(beta_local)
    print("Finished Calculating Beta\n")
    # print(beta_mu)
    
    return beta_mu

def calcBeta(XXh, mu, AAhq, thetaAq, ffh, thetaF):
    Qa = len(AAhq)
    AAh = np.zeros_like(AAhq[0])
    
    for q in range(Qa):
        AAh = AAh + thetaAq[q](mu) * AAhq[q]
    
    M = scipy.sparse.linalg.spsolve(XXh, AAh) 
    _, beta, _ = scipy.sparse.linalg.svds(M)
    minEig = beta[-1]
    
    return minEig

--- 
The actual Greedy Algorithm

In [11]:
def greetyAlgorithm(XXh, AAh, thetaA, ffh, thetaF, beta_mu,
                    xi_train:np.array, mu_0:np.array, N_max:int=5, epsilon_g:float=1e-3):   
    
    print("Greety Algorithm - Start \n")
    N = 0
    delta_0 = epsilon_g + 1
    
    xi_g   = np.array([])
    V_base = np.array([])
    
    mu_N = mu_0
    delta_N = delta_0  
    
    # method = "G-RB"   
    method = "LS-RB"
    
    print("Start Iterative Method\n")
    while N < N_max and delta_N > epsilon_g:   
        print(f"Start Run {N}:")
        N = N+1
        
        uh_N = solveHFsystem(AAh, ffh, thetaA, thetaF, mu_N)     
        zeta_N = gramSchmidt(V_base, uh_N, XXh)
        
        V_base = addZeta(V_base, zeta_N)
        xi_g = addMu(xi_g, mu_N)   
        
        
        AAN, ffN = projectSystem(AAh, ffh, V_base, XXh, method)
        C, d, EE = offlineResidual(AAh, ffh, XXh, V_base)
   
    
        Delta = np.zeros(xi_train.shape[0])
        for i in range(xi_train.shape[0]):
            mu = xi_train[i]
            uN = solveRBsystem(AAN, ffN, thetaA, thetaF, mu, method)
            Delta[i] = errorEstimate(C, d, EE, thetaA, thetaF, uN, mu, beta_mu[i])
        
        
        idxMax = np.argmax(Delta)
        delta_N = Delta[idxMax]
        mu_N = xi_train[idxMax, :]        
        
        print("New Mu:")
        print(mu_N)
        print("Error:")
        print(delta_N)
        print()
        
        
    return xi_g

In [12]:
######################################################
# --- Greety Algorithm --- #
######################################################
# 0. Choose ρ ∈ (0, 1]
# 1. Choose μ1 s.t. ||uh(μ)|| ≥ ρ max ||uh(μ)||
# 2. Given μ1, ..., μ[N-1]
#     i. V[N-1] = span{uh(μ1), ..., uh(μ[N-1])}
#    ii. Choose μN s.t. d(uh(μN), V[N-1]) ≥ ρ max d(uh(μ), V[N-1])
# 3. Iterate until  max d(uh(μ), VN) < ε
######################################################


# Parameters
mu_0 = [6, 6, 6, 125]
N_max = 50
epsilon_g = 1e-3


# Overhead
calcPDE_NS, XX, mesh = overhead()
print("\nCreated Matrix XX")

AA = dP.calcAA(mesh)
thetaA = dP.calcThetaA()

ff = dP.calcFF(mesh)
thetaF = dP.calcThetaF()
print("Defined Problemset")

greetyA = lambda beta_mu, xi_train, mu_0, N_max=N_max, epsilon_g=epsilon_g: greetyAlgorithm(XX, AA, thetaA, ff, thetaF, beta_mu,
                                                                                      xi_train, mu_0, N_max, epsilon_g)

Info    : Reading './mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading './mesh/mixer.msh'

Created Matrix XX
Defined Problemset


In [13]:
n_train = 100

sampler = qmc.LatinHypercube(d=4)
xi_train = np.array([12, 12, 12, 599]) * sampler.random(n=n_train) + np.array([0,0,0,1])
print("Created Initial Training Set\n\n")

globalBeta = calcGlobalBeta(xi_train, XX, AA, thetaA, ff, thetaF);

Created Initial Training Set


Start Calculating Beta


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [02:35<00:00,  1.55s/it]

Finished Calculating Beta



In [14]:
mu_0 = np.array(mu_0)
mu = greetyA(globalBeta, xi_train, mu_0)

Greety Algorithm - Start 

Start Iterative Method

Start Run 0:
New Mu:
[ 11.87693063   3.64883796   0.70129957 514.39344412]
Error:
0.09656574053598259

Start Run 1:
New Mu:
[9.20480772e+00 1.17549089e+01 6.12736929e-02 3.89185027e+02]
Error:
0.06723303740884909

Start Run 2:
New Mu:
[ 11.47634106   2.87810749   5.91981628 190.97470107]
Error:
5.792511121716269e-09



<font color='Red'> <b>ToDo:</b> </font> <br>
Construct the Snapshotmatrix according to the parameter set of the Greedy Algorithm. Investigate the SVD of this matrix.